# Regression of Discussion Characteristics on CAR

In [1]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      StataNow 19.5
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2025 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-782-8272        https://www.stata.com
                                   979-696-4600        service@stata.com

Stata license: Unlimited-user 2-core network, expiring  9 Sep 2026
Serial number: 501909305069
  Licensed to: 罗奕辰
               UCL

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


In [2]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"


. 
. clear all

. set more off

. set varabbrev off

. version 18

. 
. * Paths
. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. 


In [3]:
%%stata
foreach stage in created end {
    import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv", clear

    * Parse date
    gen day = date(date, "YMD")
    format day %td
    gen year = year(day)

    * Ensure index is numeric (safeguard if it came in as string)
    capture confirm numeric variable index
    if _rc {
        destring index, replace ignore("[]")
    }

    * Encode categorical variables
    encode gecko_id, gen(token)
    encode space,    gen(dao)

    * Define windows and specs
    local windows 2 5
    local vars    support professionalism objectiveness unanimity

    eststo clear

    foreach w of local windows {
        preserve
        keep if index == `w'

        foreach v of local vars {
            reghdfe car_`w' `v', absorb(dao year) vce(cluster token)
            eststo car`w'_`v'
        }

        restore
    }

    * Export a single big LaTeX table with 8 models (2 windows × 4 specs)
    esttab                                                          ///
        car2_support car2_professionalism car2_objectiveness car2_unanimity ///
        car5_support car5_professionalism car5_objectiveness car5_unanimity ///
        using "$TABLES/reg_car_discussions_`stage'.tex", replace          ///
        se star(* 0.10 ** 0.05 *** 0.01) label nogaps               ///
        mtitles("Support" "Professionalism" "Objectiveness" "Unanimity"  ///
                "Support" "Professionalism" "Objectiveness" "Unanimity") ///
        mgroups("CAR[-2,2]" "CAR[-5,5]",                            ///
            span                                                   ///
            pattern(1 0 0 0 1 0 0 0)                               ///
            prefix(\multicolumn{@span}{c}{)                        ///
            suffix(}))                                             ///
        stats(N r2_a, fmt(0 3) labels("N" "Adj. R²"))
}


. foreach stage in created end {
  2.     import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv"
> , clear
  3. 
.     * Parse date
.     gen day = date(date, "YMD")
  4.     format day %td
  5.     gen year = year(day)
  6. 
.     * Ensure index is numeric (safeguard if it came in as string)
.     capture confirm numeric variable index
  7.     if _rc {
  8.         destring index, replace ignore("[]")
  9.     }
 10. 
.     * Encode categorical variables
.     encode gecko_id, gen(token)
 11.     encode space,    gen(dao)
 12. 
.     * Define windows and specs
.     local windows 2 5
 13.     local vars    support professionalism objectiveness unanimity
 14. 
.     eststo clear
 15. 
.     foreach w of local windows {
 16.         preserve
 17.         keep if index == `w'
 18. 
.         foreach v of local vars {
 19.             reghdfe car_`w' `v', absorb(dao year) vce(cluster token)
 20.             eststo car`w'_`v'
 21.         }
 22. 
.         restore
 23.   